In [2]:
import numpy as np
import os
from unicodedata import category, normalize
import pickle

In [3]:
directories = os.listdir("aksharantar_sampled/hin/")
train_data, val_data, test_data = [],[],[]

for i in open("aksharantar_sampled/hin/"+directories[0], encoding='utf-8'):
    test_data.append([i.split(",")[0],i.split(",")[1][:-1]])

for i in open("aksharantar_sampled/hin/"+directories[1], encoding='utf-8'):
    train_data.append([i.split(",")[0],i.split(",")[1][:-1]])

for i in open("aksharantar_sampled/hin/"+directories[2], encoding='utf-8'):
    val_data.append([i.split(",")[0],i.split(",")[1][:-1]])

print(len(test_data))
print(len(train_data))
print(len(val_data))

def hindi_chars(hindi_word):

    return [c for c in normalize('NFD', hindi_word) if category(c) != 'Mn']

def english_chars(english_word):

    return [c for c in english_word]

def one_hot(x, n):

    encoding = np.zeros(n)
    encoding[x] = 1
    return encoding

def generate_char_to_idx():
    
    en_start, en_end = 0x0061, 0x007A # ASCII characters (lowercase only)
    hi_start, hi_end = 0x0900, 0x0965 # Devanagari characters

    char_to_idx = {}

    for i, char in enumerate(range(en_start, en_end+1)):
        char_to_idx[chr(char)] = len(char_to_idx)

    for i, char in enumerate(range(hi_start, hi_end+1)):
        char_to_idx[chr(char)] = len(char_to_idx)

    return char_to_idx

char_to_idx = generate_char_to_idx()
char_to_idx['/start'] = len(char_to_idx)
char_to_idx['/end'] = len(char_to_idx)
char_to_idx['/pad']  = len(char_to_idx)

def word_to_idx(data):

    idx = 0

    for eng_word, hin_word in data:

        eng_sequence = [char_to_idx[char] for char in eng_word]
        hin_sequence = [char_to_idx[char] for char in hin_word]

        eng_sequence.insert(0,char_to_idx['/start'])
        hin_sequence.insert(0,char_to_idx['/start'])

        eng_sequence.append(char_to_idx['/end'])
        hin_sequence.append(char_to_idx['/end'])

        data[idx][0] = eng_sequence
        data[idx][1] = hin_sequence

        idx+=1

    return data

train_data = word_to_idx(train_data)
test_data = word_to_idx(test_data)
val_data = word_to_idx(val_data)

print(len(char_to_idx))



4096
51200
4096
131


In [4]:
max_seq_length = 0

def set_max_seq(data):

    max_seq_length = 0

    for seq_x, seq_y in data:

        if(len(seq_x) > max_seq_length) : max_seq_length = len(seq_x)
        if(len(seq_y) > max_seq_length) : max_seq_length = len(seq_y)

    return max_seq_length

def pad(data, max_seq_length):

    idx = 0

    for seq_x, seq_y in data:

        while(len(seq_x) < max_seq_length):

            seq_x.insert(-1, char_to_idx['/pad'])

        while(len(seq_y) < max_seq_length):

            seq_y.insert(-1, char_to_idx['/pad'])

        data[idx][0] = seq_x
        data[idx][1] = seq_y

        idx+=1

    return data

max_seq_length = max(max_seq_length , set_max_seq(train_data))
max_seq_length = max(max_seq_length , set_max_seq(test_data))
max_seq_length = max(max_seq_length , set_max_seq(val_data))

print(max_seq_length)

train_data = np.array(pad(train_data, max_seq_length))
test_data = np.array(pad(test_data, max_seq_length))
val_data = np.array(pad(val_data, max_seq_length))

x_train,y_train = train_data[:,0],train_data[:,1]
x_val,y_val = val_data[:,0],val_data[:,1]
x_test,y_test = test_data[:,0],test_data[:,1]

np.savetxt("akshar_sequences/x_train.csv", np.array(x_train), delimiter=",", fmt="%s")
np.savetxt("akshar_sequences/x_test.csv", np.array(x_test), delimiter=",", fmt="%s")
np.savetxt("akshar_sequences/x_val.csv", np.array(x_val), delimiter=",", fmt="%s")
np.savetxt("akshar_sequences/y_train.csv", np.array(y_train), delimiter=",", fmt="%s")
np.savetxt("akshar_sequences/y_test.csv", np.array(y_test), delimiter=",", fmt="%s")
np.savetxt("akshar_sequences/y_val.csv", np.array(y_val), delimiter=",", fmt="%s")

28
